In [1]:
from openpyxl import load_workbook, Workbook
from datetime import datetime
wb = load_workbook('上下班时间表.xlsx')


In [2]:
sheet = wb.worksheets[0]
print(sheet)
minutes_off = 0 # 总分钟数
day_num = 0 # 上班天数

for r in sheet['C']: # 下班时间放在C列，每个时间格式为：xx点xx，如：18点30
    s=r.value
    if s is not None and s.count('点') == 1:
        t=s.split('点')
        minutes_off += int(t[0])*60 + int(t[1])
        day_num += 1

print("上班天数：", day_num-1,"天")
print("平均下班时间：", (minutes_off+day_num//2)//day_num//60, "点",
      str((minutes_off+day_num//2)//day_num % 60).rjust(2, "0"), "分")

minutes_left = 0  # 总分钟数
for r in sheet['D']:  # 离开时间放在D列
    s = r.value
    if s is not None and s.count('点') == 1:
        t = s.split('点')
        minutes_left += int(t[0])*60 + int(t[1])
print("平均离开时间：", (minutes_left+day_num//2)//day_num//60, "点",
      str((minutes_left+day_num//2)//day_num % 60).rjust(2, "0"), "分")

print("平均主动加班时长: ", (minutes_left-minutes_off+day_num//2)//day_num, "分钟")

day_min = (minutes_left-10*60*day_num+day_num//2)//day_num
print("日均上班时长: ", day_min//60, "小时", day_min%60, "分钟")

<Worksheet "孝庸基金">
上班天数： 33 天
平均下班时间： 18 点 02 分
平均离开时间： 19 点 10 分
平均主动加班时长:  68 分钟
日均上班时长:  9 小时 10 分钟


In [34]:
sheet = wb.worksheets[2]

on_minutes = [] # 上班时间（一天里的第几分钟）
off_minutes = [] # 下班时间（一天里的第几分钟）
long_hours = [] # 每天在司小时数
long_hours_filter_small = [] # 过滤掉小于 8 小时的情况（请假等）

sum_of_hours = 0 # 在公司小时数
real_work_hour_num = 2786 # 实际总上班小时数

work_day_num = 0 # 上班天数
total_day_num = 0 # 在职天数

min_day_minute = 10000 # 最短天工作分钟数
max_day_minute = 0     # 最长工作分钟数
min_day_min_row = None # 最短时长的那一行
max_day_min_row = None # 最长时长的那一行
lingchen_leave = []    # 凌晨走的天信息

begin_date = datetime(2023,3,30)
end_date = datetime(2024,6,7)
total_day_num = (end_date-begin_date).days + 1 # 在职天数

def pre_process():
    global on_minutes, off_minutes, long_hours, long_hours_filter_small, sum_of_hours, real_work_hour_num, work_day_num, total_day_num
    global min_day_minute, max_day_minute, min_day_hour, max_day_hour, lingchen_leave
    rows = []
    for r in sheet:
        if r[1].value == None:
            # print(r)
            continue
        work_day_num+=1
        row = [str(r[1].value)] + [d.value for d in r[2:]]
        # print(row)
        rows.append(row)
    # print(rows)
    print('在职天数:', total_day_num, '上班天数:', work_day_num, '占比:', round(work_day_num/total_day_num, 2))

    for i, row in enumerate(rows):
        if i<186:
            row[0] = '2023-'+row[0].replace('.', '-')
        else:
            row[0] = '2024-'+row[0].replace('.', '-')
        p = row[0].split('-')
        month = p[1].zfill(2)
        day = p[2].zfill(2)
        row[0] = f"{p[0]}-{month}-{day}"
    sorted_rows = rows[:186] + list(reversed(rows[186:]))

    for s in sorted_rows:
        print(s)
    cwb = Workbook()
    cws = cwb.active
    for row in sorted_rows:
        cws.append(row)
    # cwb.save("tmp.xlsx")

def on_process():
    global on_minutes, off_minutes, long_hours, long_hours_filter_small, sum_of_hours, real_work_hour_num, work_day_num, total_day_num
    global min_day_minute, max_day_minute, min_day_hour, max_day_hour, lingchen_leave
    rows = []
    for r in sheet:
        if r[1].value == None:
            continue
        work_day_num+=1
        row =[d.value for d in r[1:]]
        # print(row)
        rows.append(row)
    # print(rows)

    
    for r in rows:
        bmin = r[1]*60+r[2]
        emin = r[3]*60+r[4]
        this_day_min = emin - bmin
        if bmin > emin: # 凌晨才走的只有24年5月有两天，之前都是直接写的23:59
            emin += 24*60
            this_day_min = emin - bmin
        if emin >= 23*60+30:
            lingchen_leave.append([r, bmin, emin])
            # print(r, bmin, emin)
        if this_day_min<min_day_minute:
            min_day_min_row = r
            min_day_minute = this_day_min
        if this_day_min>max_day_minute:
            max_day_min_row = r
            max_day_minute = this_day_min
        if this_day_min>8*60:
            long_hours_filter_small.append(round(this_day_min/60, 2))
        on_minutes.append(bmin)
        off_minutes.append(emin)
        long_hours.append(round(this_day_min/60, 2))
    print(on_minutes)
    print(off_minutes)
    print('每天在司小时数', long_hours, '\n')
    sum_of_hours = int(sum(long_hours))
    print('上班天数:', work_day_num, '在职天数:', total_day_num, '占比:', round(work_day_num/total_day_num, 3))
    print('总在司小时数:', sum_of_hours, '实际工作小时数(番茄Todo):', real_work_hour_num , '平均每天在司小时数:', round(sum_of_hours/work_day_num,1))
    print('大于8小时的正常工作天数:', len(long_hours_filter_small), '正常平均每天上班小时数:', round(sum(long_hours_filter_small)/len(long_hours_filter_small),1))
    print('最长工作的那天:', max_day_minute/60, '小时', max_day_min_row)
    print('最短工作的那天:', round(min_day_minute/60,1), '小时', min_day_min_row, '\n')
    print('23:30 后离开公司的天数:',len(lingchen_leave), '(凌晨才走的只有24年5月有两天，之前都是直接写的23:59)')
    for ll in lingchen_leave:
        print(ll)
    cwb = Workbook()
    cws = cwb.active
    for row in long_hours:
        cws.append([row])
    cwb.save("tmp.xlsx")

on_process()

[559, 573, 569, 572, 569, 570, 589, 570, 568, 570, 572, 569, 570, 563, 842, 577, 573, 570, 566, 573, 567, 588, 571, 572, 571, 564, 571, 561, 565, 572, 583, 571, 584, 563, 575, 576, 575, 570, 574, 570, 576, 577, 574, 582, 580, 569, 577, 563, 590, 580, 575, 580, 592, 581, 578, 565, 576, 567, 575, 570, 575, 577, 574, 575, 577, 580, 573, 571, 580, 578, 574, 575, 569, 569, 569, 575, 576, 560, 573, 577, 572, 573, 590, 572, 579, 573, 580, 578, 576, 573, 574, 571, 574, 575, 577, 582, 546, 581, 578, 577, 576, 574, 584, 585, 576, 593, 585, 586, 578, 583, 582, 583, 578, 582, 581, 573, 576, 577, 575, 585, 577, 585, 578, 580, 581, 594, 587, 583, 591, 582, 580, 574, 589, 590, 586, 587, 581, 585, 593, 589, 585, 589, 585, 582, 585, 584, 584, 586, 595, 589, 586, 598, 595, 588, 591, 593, 590, 584, 585, 596, 591, 593, 592, 594, 590, 840, 588, 588, 588, 586, 591, 592, 596, 590, 592, 603, 590, 591, 589, 587, 585, 588, 589, 591, 589, 594, 588, 590, 591, 561, 589, 593, 590, 590, 585, 583, 591, 587, 591, 588,